In [7]:
import numpy as np 
import csv
import matplotlib.pyplot as plt
import os
from copy import deepcopy
np.random.seed(40)
percent=0.4
path='../数据'
#files=os.listdir(path)
files=['ib1000_2.csv','ib600_2.csv','ib800_2.csv','n1000_3_2.csv','n600_3_2.csv',
       'n800_3_2.csv','ob1000_2.csv', 'ob600_2.csv','ob800_2.csv', 'tb1000_2.csv', 
       'tb600_2.csv','tb800_2.csv']
temp_data=[]
temp_data_all=[]
for item in files:
 
   if 'csv' in item:
       root=os.path.join(path,item)
       csv_reader = csv.reader(open(root, encoding='utf-8'))
       for row in csv_reader:
           temp_data.append(row[0])
       temp_data=np.asarray(temp_data)
       temp_data=temp_data.astype(np.float64)
       if temp_data.shape[0]==500500:        
           temp_data_all.append(temp_data[:500000])
       else:
#           temp_data=np.reshape(temp_data,(3,500500))
           temp_data_all.append(temp_data[:1500000])
       temp_data=[]
train_data=[]
test_data=[]
for item in temp_data_all:
    item=item.reshape(-1,10000)
    train_data.append(item[:-int(item.shape[0]*percent)])
    test_data.append(item[-int(item.shape[0]*percent):])

a=deepcopy(train_data[0])
for index in range(1,len(train_data)):
    a=np.concatenate((a,train_data[index]))
train_data=deepcopy(a)

a=deepcopy(test_data[0])
for index in range(1,len(test_data)):
    a=np.concatenate((a,test_data[index]))
test_data=deepcopy(a)
   
#data=np.reshape(data,(18,500000))
#data=np.reshape(data,(900,10000))
#data1=2*(data-np.min(data,axis=0))/(np.max(data,axis=0)-np.min(data,axis=0))-1
#data1=np.expand_dims(data1,axis=2)
#normalize = lambda series: 2*(series - np.min(series)) / (np.max(series) - np.min(series))-1
#data=np.asarray(list(map(normalize, data)))

#label=[0,0,0,1,1,1,1,1,1,1,1,1,2,2,2,3,3,3]
label0=np.zeros((int(150*(1-percent)),1))
label1=np.ones((int(450*(1-percent)),1))
label2=np.zeros((int(150*(1-percent)),1))
label2[:,:]=2
label3=np.zeros((int(150*(1-percent)),1))
label3[:,:]=3
train_label=np.concatenate((label0,label1,label2,label3))

label0=np.zeros((int(150*percent),1))
label1=np.ones((int(450*percent),1))
label2=np.zeros((int(150*percent),1))
label2[:,:]=2
label3=np.zeros((int(150*percent),1))
label3[:,:]=3
test_label=np.concatenate((label0,label1,label2,label3))

from keras.utils import to_categorical
train_label=to_categorical(train_label)
test_label=to_categorical(test_label)

from keras.models import Model
from keras.layers import Input,Dense,Flatten,Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPool1D
from keras import optimizers
#from sklearn.cross_validation import train_test_split


#mean_value=np.mean(x_train,axis=0)
#max_value=np.max(x_train,axis=0)
#min_value=np.min(x_train,axis=0)
#x_train=(x_train-mean_value)/(max_value-min_value)
#x_test=(x_test-mean_value)/(max_value-min_value)
#from sklearn import preprocessing
#scaler= preprocessing.StandardScaler().fit(x_train)
#train_data=scaler.transform(x_train)
#test_data=scaler.transform(x_test)

train_data=np.expand_dims(train_data,axis=2)
test_data=np.expand_dims(test_data,axis=2)

input=Input(shape=(10000,1))
conv1=Conv1D(32,kernel_size=1000,strides=10,activation='tanh')(input)
pool1=MaxPool1D(pool_size=2)(conv1)
conv2=Conv1D(64,kernel_size=2,strides=1,activation='tanh')(pool1)
pool2=MaxPool1D(pool_size=2)(conv2)
#conv3=Conv1D(64,kernel_size=2,activation='relu')(pool2)
#pool3=MaxPool1D(pool_size=2)(conv3)



flat1=Flatten(name='flatten')(pool2)

dense2=Dense(50,activation='tanh')(flat1)
dense3=Dropout(0.5)(dense2)
#dense4=Dense(50,activation='relu')(dense2)

output=Dense(4,activation='softmax',name='result')(dense3)
model=Model(inputs=input,outputs=output)


optimizer = optimizers.adam(lr=0.001)
model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy'])
model.summary()
history=model.fit(train_data,train_label, epochs=500,validation_data=(test_data,test_label))
dense1_layer_model = Model(inputs=model.input,  
                           outputs=model.get_layer('result').output)
predict = dense1_layer_model.predict(train_data)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 10000, 1)          0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 901, 32)           32032     
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 450, 32)           0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 449, 64)           4160      
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 224, 64)           0         
_________________________________________________________________
flatten (Flatten)            (None, 14336)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 50)                716850    
__________

Epoch 50/500
540/540 [==============================] - 0s 175us/step - loss: 9.8645e-04 - acc: 1.0000 - val_loss: 0.6122 - val_acc: 0.8528
Epoch 51/500
540/540 [==============================] - 0s 172us/step - loss: 9.9089e-04 - acc: 1.0000 - val_loss: 0.6338 - val_acc: 0.8500
Epoch 52/500
540/540 [==============================] - 0s 166us/step - loss: 0.0010 - acc: 1.0000 - val_loss: 0.6486 - val_acc: 0.8417
Epoch 53/500
540/540 [==============================] - 0s 154us/step - loss: 7.3546e-04 - acc: 1.0000 - val_loss: 0.6446 - val_acc: 0.8472
Epoch 54/500
540/540 [==============================] - 0s 154us/step - loss: 9.5115e-04 - acc: 1.0000 - val_loss: 0.6290 - val_acc: 0.8583
Epoch 55/500
540/540 [==============================] - 0s 152us/step - loss: 9.4242e-04 - acc: 1.0000 - val_loss: 0.6274 - val_acc: 0.8556
Epoch 56/500
540/540 [==============================] - 0s 156us/step - loss: 7.0705e-04 - acc: 1.0000 - val_loss: 0.6262 - val_acc: 0.8528
Epoch 57/500
540/540 [==

540/540 [==============================] - 0s 163us/step - loss: 3.5638e-04 - acc: 1.0000 - val_loss: 0.7293 - val_acc: 0.8417
Epoch 109/500
540/540 [==============================] - 0s 154us/step - loss: 2.6250e-04 - acc: 1.0000 - val_loss: 0.7251 - val_acc: 0.8417
Epoch 110/500
540/540 [==============================] - 0s 154us/step - loss: 3.6304e-04 - acc: 1.0000 - val_loss: 0.7201 - val_acc: 0.8472
Epoch 111/500
540/540 [==============================] - 0s 150us/step - loss: 2.8240e-04 - acc: 1.0000 - val_loss: 0.7237 - val_acc: 0.8500
Epoch 112/500
540/540 [==============================] - 0s 151us/step - loss: 2.8363e-04 - acc: 1.0000 - val_loss: 0.7263 - val_acc: 0.8500
Epoch 113/500
540/540 [==============================] - 0s 153us/step - loss: 2.8397e-04 - acc: 1.0000 - val_loss: 0.7259 - val_acc: 0.8472
Epoch 114/500
540/540 [==============================] - 0s 153us/step - loss: 3.2034e-04 - acc: 1.0000 - val_loss: 0.7286 - val_acc: 0.8500
Epoch 115/500
540/540 [====

540/540 [==============================] - 0s 159us/step - loss: 1.3851e-04 - acc: 1.0000 - val_loss: 0.7783 - val_acc: 0.8556
Epoch 167/500
540/540 [==============================] - 0s 151us/step - loss: 1.5981e-04 - acc: 1.0000 - val_loss: 0.7793 - val_acc: 0.8556
Epoch 168/500
540/540 [==============================] - 0s 154us/step - loss: 1.9224e-04 - acc: 1.0000 - val_loss: 0.7710 - val_acc: 0.8556
Epoch 169/500
540/540 [==============================] - 0s 153us/step - loss: 8.4125e-05 - acc: 1.0000 - val_loss: 0.7655 - val_acc: 0.8556
Epoch 170/500
540/540 [==============================] - 0s 151us/step - loss: 1.4251e-04 - acc: 1.0000 - val_loss: 0.7644 - val_acc: 0.8556
Epoch 171/500
540/540 [==============================] - 0s 154us/step - loss: 1.5923e-04 - acc: 1.0000 - val_loss: 0.7620 - val_acc: 0.8583
Epoch 172/500
540/540 [==============================] - 0s 152us/step - loss: 2.0199e-04 - acc: 1.0000 - val_loss: 0.7501 - val_acc: 0.8667
Epoch 173/500
540/540 [====

540/540 [==============================] - 0s 150us/step - loss: 6.5102e-05 - acc: 1.0000 - val_loss: 0.8647 - val_acc: 0.8444
Epoch 225/500
540/540 [==============================] - 0s 151us/step - loss: 9.2122e-05 - acc: 1.0000 - val_loss: 0.8677 - val_acc: 0.8444
Epoch 226/500
540/540 [==============================] - 0s 150us/step - loss: 1.2226e-04 - acc: 1.0000 - val_loss: 0.8594 - val_acc: 0.8528
Epoch 227/500
540/540 [==============================] - 0s 153us/step - loss: 1.0874e-04 - acc: 1.0000 - val_loss: 0.8550 - val_acc: 0.8472
Epoch 228/500
540/540 [==============================] - 0s 152us/step - loss: 4.6320e-05 - acc: 1.0000 - val_loss: 0.8496 - val_acc: 0.8528
Epoch 229/500
540/540 [==============================] - 0s 149us/step - loss: 1.0948e-04 - acc: 1.0000 - val_loss: 0.8526 - val_acc: 0.8556
Epoch 230/500
540/540 [==============================] - 0s 150us/step - loss: 5.3092e-05 - acc: 1.0000 - val_loss: 0.8610 - val_acc: 0.8556
Epoch 231/500
540/540 [====

540/540 [==============================] - 0s 153us/step - loss: 5.5819e-05 - acc: 1.0000 - val_loss: 0.8698 - val_acc: 0.8583
Epoch 283/500
540/540 [==============================] - 0s 150us/step - loss: 4.4581e-05 - acc: 1.0000 - val_loss: 0.8689 - val_acc: 0.8556
Epoch 284/500
540/540 [==============================] - 0s 152us/step - loss: 5.5945e-05 - acc: 1.0000 - val_loss: 0.8772 - val_acc: 0.8556
Epoch 285/500
540/540 [==============================] - 0s 152us/step - loss: 4.1307e-05 - acc: 1.0000 - val_loss: 0.8853 - val_acc: 0.8556
Epoch 286/500
540/540 [==============================] - 0s 154us/step - loss: 3.7722e-05 - acc: 1.0000 - val_loss: 0.8896 - val_acc: 0.8556
Epoch 287/500
540/540 [==============================] - 0s 152us/step - loss: 4.7360e-05 - acc: 1.0000 - val_loss: 0.8817 - val_acc: 0.8556
Epoch 288/500
540/540 [==============================] - 0s 153us/step - loss: 9.8614e-05 - acc: 1.0000 - val_loss: 0.8988 - val_acc: 0.8528
Epoch 289/500
540/540 [====

540/540 [==============================] - 0s 154us/step - loss: 3.5796e-05 - acc: 1.0000 - val_loss: 0.9267 - val_acc: 0.8556
Epoch 341/500
540/540 [==============================] - 0s 153us/step - loss: 3.8520e-05 - acc: 1.0000 - val_loss: 0.9236 - val_acc: 0.8528
Epoch 342/500
540/540 [==============================] - 0s 153us/step - loss: 8.0305e-05 - acc: 1.0000 - val_loss: 0.9348 - val_acc: 0.8556
Epoch 343/500
540/540 [==============================] - 0s 153us/step - loss: 2.8665e-05 - acc: 1.0000 - val_loss: 0.9390 - val_acc: 0.8556
Epoch 344/500
540/540 [==============================] - 0s 152us/step - loss: 4.2976e-05 - acc: 1.0000 - val_loss: 0.9516 - val_acc: 0.8556
Epoch 345/500
540/540 [==============================] - 0s 153us/step - loss: 3.2312e-05 - acc: 1.0000 - val_loss: 0.9582 - val_acc: 0.8556
Epoch 346/500
540/540 [==============================] - 0s 155us/step - loss: 3.2697e-05 - acc: 1.0000 - val_loss: 0.9714 - val_acc: 0.8556
Epoch 347/500
540/540 [====

540/540 [==============================] - 0s 153us/step - loss: 3.5206e-05 - acc: 1.0000 - val_loss: 0.9660 - val_acc: 0.8556
Epoch 399/500
540/540 [==============================] - 0s 153us/step - loss: 1.2810e-05 - acc: 1.0000 - val_loss: 0.9723 - val_acc: 0.8556
Epoch 400/500
540/540 [==============================] - 0s 153us/step - loss: 2.3517e-05 - acc: 1.0000 - val_loss: 0.9708 - val_acc: 0.8500
Epoch 401/500
540/540 [==============================] - 0s 153us/step - loss: 1.1692e-05 - acc: 1.0000 - val_loss: 0.9666 - val_acc: 0.8472
Epoch 402/500
540/540 [==============================] - 0s 152us/step - loss: 1.0838e-05 - acc: 1.0000 - val_loss: 0.9592 - val_acc: 0.8472
Epoch 403/500
540/540 [==============================] - 0s 155us/step - loss: 3.3936e-05 - acc: 1.0000 - val_loss: 0.9672 - val_acc: 0.8472
Epoch 404/500
540/540 [==============================] - 0s 153us/step - loss: 1.7479e-05 - acc: 1.0000 - val_loss: 0.9756 - val_acc: 0.8472
Epoch 405/500
540/540 [====

540/540 [==============================] - 0s 154us/step - loss: 6.0841e-06 - acc: 1.0000 - val_loss: 0.9828 - val_acc: 0.8583
Epoch 457/500
540/540 [==============================] - 0s 153us/step - loss: 9.3347e-06 - acc: 1.0000 - val_loss: 0.9853 - val_acc: 0.8611
Epoch 458/500
540/540 [==============================] - 0s 152us/step - loss: 1.1572e-05 - acc: 1.0000 - val_loss: 0.9877 - val_acc: 0.8583
Epoch 459/500
540/540 [==============================] - 0s 154us/step - loss: 7.7594e-06 - acc: 1.0000 - val_loss: 0.9911 - val_acc: 0.8639
Epoch 460/500
540/540 [==============================] - 0s 154us/step - loss: 8.0566e-06 - acc: 1.0000 - val_loss: 0.9920 - val_acc: 0.8639
Epoch 461/500
540/540 [==============================] - 0s 156us/step - loss: 1.1504e-05 - acc: 1.0000 - val_loss: 0.9960 - val_acc: 0.8639
Epoch 462/500
540/540 [==============================] - 0s 152us/step - loss: 9.4133e-06 - acc: 1.0000 - val_loss: 1.0016 - val_acc: 0.8639
Epoch 463/500
540/540 [====

In [6]:
print (len(test_label))
print (len(test_label)*0.9630)


270
260.01
